## Requirements

Please install cuda on your device if you have a GPU available.  
This line in the miniconda prompt worked for me: conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0  
You can refer to this: https://stackoverflow.com/questions/45662253/can-i-run-keras-model-on-gpu  
And also this: https://www.tensorflow.org/install/pip#linux

## Imports

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0" # Makes visible cuda devices, -1 otherwise

In [ ]:
from helpers import *
from models import *

import numpy as np

import math
import random


import skimage.io as io
import skimage.transform as trans

from tqdm import tqdm

from sklearn.model_selection import train_test_split

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import tensorflow as tf
print(f'This value has to be at most 2.10.x ---> {tf.__version__}')

In [ ]:
# This will print logs and cannot be disabled (except restart). Run only to check that GPU is enabled
#tf.debugging.set_log_device_placement(True)

In [ ]:
# Make use of GPU
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# with tf.device('/cpu:0'): Force CPU utilization instead of GPU
# This code should run on the GPU, you can see it by uncommenting the code in the previous cell
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)
print(c)

## Constants

In [ ]:
%run models.py # If you have error of XXX not found below, try this
%run helpers.py

In [ ]:
TRAIN_DIRECTORY_PATH = './training/'
TRAIN_IMAGES_PATH = TRAIN_DIRECTORY_PATH + 'images/'
TRAIN_GROUNDTRUTH_PATH = TRAIN_DIRECTORY_PATH + 'groundtruth/'

NEW_TRAIN_DIRECTORY_PATH = './new_training/'
NEW_TRAIN_IMAGES_PATH = NEW_TRAIN_DIRECTORY_PATH + 'images/'
NEW_TRAIN_GROUNDTRUTH_PATH = NEW_TRAIN_DIRECTORY_PATH + 'groundtruth/'

TEST_DIRECTORY_PATH = './test_set_images/'
TEST_IMAGES_PATH = [TEST_DIRECTORY_PATH + "test_" + str(i) + "/" for i in range(1,51)]

PATCH_SIZE = 96
NUMBER_NEW_TRAINING_TO_TAKE = 0 # Used
NUMBER_CHANNELS_INPUT = 3
BATCH_SIZE = 64 # Put 16 to avoid burning your laptop

MODEL_FUNCTION = fat_unet # Just implement your model in models.py and change this
MODEL = MODEL_FUNCTION((PATCH_SIZE, PATCH_SIZE, NUMBER_CHANNELS_INPUT), verbose = False)

CHECKPOINT_PATH = "./check_points/" + str(MODEL_FUNCTION.__name__)
SAVE_MODEL_PATH = "./models/" + str(MODEL_FUNCTION.__name__) + ".h5"

RANDOM = np.random.randint(69)

tf.random.set_seed(RANDOM)

In [ ]:
# Load model from local models folder
#MODEL = tf.keras.models.load_model(SAVE_MODEL_PATH, custom_objects={'get_f1': get_f1})

## Load data

### Train

In [ ]:
train_images = []

for file in tqdm(os.listdir(TRAIN_IMAGES_PATH), total=len(os.listdir(TRAIN_IMAGES_PATH))):
    img = plt.imread(TRAIN_IMAGES_PATH + file)
    img_split = split_into_patches(img, PATCH_SIZE)
    train_images.append(img_split)

# New images
new_train_images = []
for num, file in enumerate(os.listdir(NEW_TRAIN_IMAGES_PATH)):
    if num == NUMBER_NEW_TRAINING_TO_TAKE:
        break
    img = plt.imread(NEW_TRAIN_IMAGES_PATH + file)
    img_split = split_into_patches(img, PATCH_SIZE)
    new_train_images.append(img_split)

train_images = np.array(train_images)
new_train_images = np.array(new_train_images)

# Below, this merges the first two dimensions. Instead of having x elements of y patches, we have x*y patches.
train_images = combine_dims(train_images, start = 0, count = 2)
new_train_images = combine_dims(new_train_images, start = 0, count = 2)
print(f'Base train shape: {train_images.shape}')
print(f'New train shape: {new_train_images.shape}')

# Add new training
if NUMBER_NEW_TRAINING_TO_TAKE:
    train_images = np.concatenate((train_images, new_train_images))
    print(f'Concatenated train shape: {train_images.shape}')

In [ ]:
train_labels = []

for file in tqdm(os.listdir(TRAIN_GROUNDTRUTH_PATH),total=len(os.listdir(TRAIN_GROUNDTRUTH_PATH))):
    img = plt.imread(TRAIN_GROUNDTRUTH_PATH + file)
    img_split = split_into_patches(img, PATCH_SIZE)
    train_labels.append(img_split)

# New images
new_train_labels = []
for num, file in enumerate(os.listdir(NEW_TRAIN_GROUNDTRUTH_PATH)):
    if num == NUMBER_NEW_TRAINING_TO_TAKE:
        break
    img = plt.imread(NEW_TRAIN_GROUNDTRUTH_PATH + file)
    img_split = split_into_patches(img, PATCH_SIZE)
    new_train_labels.append(img_split)
    
train_labels = np.array(train_labels)
new_train_labels = np.array(new_train_labels)

train_labels = combine_dims(train_labels, start = 0, count = 2)
new_train_labels = combine_dims(new_train_labels, start = 0, count = 2)
# Below, this adds a dimension at the end, such that the image is of size x*x*1, where 1 is the grayscale value of the pixel
train_labels = train_labels[:, :, :, np.newaxis]
if NUMBER_NEW_TRAINING_TO_TAKE:
    new_train_labels = new_train_labels[:, :, :, np.newaxis]
print(train_labels.shape)
print(new_train_labels.shape)

# Add new training
if NUMBER_NEW_TRAINING_TO_TAKE:
    train_labels = np.concatenate((train_labels, new_train_labels))
    print(train_labels.shape)

### Test

In [ ]:
test_images = []
test_ids = []

for directory in tqdm(TEST_IMAGES_PATH, total=len(TEST_IMAGES_PATH)):
    for file in os.listdir(directory):
        test_ids.append(file)
        img = plt.imread(directory + file)
        img_split = split_into_patches(img, PATCH_SIZE)
        test_images.append(img_split)

test_images = np.array(test_images)
# Below, this merges the first two dimensions. Instead of having x elements of y patches, we have x*y patches.
test_images = combine_dims(test_images, start = 0, count = 2)
print(test_images.shape)

test_ids = [x.split(".")[0] for x in test_ids]

### Split for validation

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size = 0.20, random_state = RANDOM)
print(X_train.shape)

## Call model

In [ ]:
callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs'),
        tf.keras.callbacks.ModelCheckpoint(filepath=CHECKPOINT_PATH, save_weights_only=True, verbose=1, save_best_only=True)]

In [ ]:
#MODEL = tf.keras.models.load_model(SAVE_MODEL_PATH, custom_objects={'get_f1': get_f1}) # Once you run the model once, you can train more by running this cell again
train_gen = DataGenerator(X_train, y_train, BATCH_SIZE)
test_gen = DataGenerator(X_test, y_test, BATCH_SIZE)

MODEL.fit(train_gen, verbose=True, epochs=100, validation_data=test_gen, shuffle=True, callbacks=callbacks)
MODEL.save(SAVE_MODEL_PATH)

### Instead of running the model you can fetch it from the file

In [ ]:
MODEL.load_weights(CHECKPOINT_PATH) #Loads best model
#print(MODEL.weights)

### Sanity checks

In [ ]:
patch_you_want_to_see = 1
pred_threshold = 0.2

fig, ax = plt.subplots(1,3)

ax[0].imshow(X_train[patch_you_want_to_see])
ax[0].title.set_text('Train image')
ax[1].imshow(y_train[patch_you_want_to_see], cmap="gray")
ax[1].title.set_text('Groundtruth')

prediction = MODEL.predict(X_train[patch_you_want_to_see][np.newaxis, :, :, :]) # Need to add an axis in front as mode expects batch
prediction = (prediction > pred_threshold).astype(np.uint8) # Transforms continuous values into 0-1

ax[2].imshow(prediction[0], cmap="gray")
ax[2].title.set_text('Prediction')

### Check test images

In [ ]:
image_you_want_to_see = 1
patch_side_len = 7 # DEPENDS ON SIZE OF IMAGE AND PATCHSIZE
pred_threshold = 0.5

test_image_side_len = patch_side_len * PATCH_SIZE
reconstructed_image = np.zeros((test_image_side_len, test_image_side_len, 3))
reconstructed_gt = np.zeros((test_image_side_len, test_image_side_len, 1))

In [ ]:
fig, ax = plt.subplots(1,2)
for i in range(patch_side_len*image_you_want_to_see,patch_side_len*image_you_want_to_see + patch_side_len):
    for j in range(patch_side_len):
        reconstructed_image[(i-patch_side_len*image_you_want_to_see)*PATCH_SIZE : (i-patch_side_len*image_you_want_to_see)*PATCH_SIZE + PATCH_SIZE, j*PATCH_SIZE : j*PATCH_SIZE + PATCH_SIZE] = test_images[patch_side_len*i + j]

ax[0].imshow(reconstructed_image)

for i in range(patch_side_len*image_you_want_to_see,patch_side_len*image_you_want_to_see + patch_side_len):
    for j in range(patch_side_len):
        prediction = MODEL.predict(test_images[patch_side_len*i + j][np.newaxis, :, :, :], verbose = False)
        prediction = (prediction > pred_threshold).astype(np.uint8)
        reconstructed_gt[(i-patch_side_len*image_you_want_to_see)*PATCH_SIZE : (i-patch_side_len*image_you_want_to_see)*PATCH_SIZE + PATCH_SIZE, j*PATCH_SIZE : j*PATCH_SIZE + PATCH_SIZE] = prediction[0]

ax[1].imshow(reconstructed_gt, cmap='gray')

## Save submission

In [ ]:
#MODEL = tf.keras.models.load_model(SAVE_MODEL_PATH, custom_objects={'get_f1': get_f1})

In [ ]:
test_predictions = MODEL.predict(test_images)

In [ ]:
submission_thres = 0.2
prediction_to_csv(test_predictions, test_ids, PATCH_SIZE, submission_thres)